In [3]:
import tensorflow as tf
import numpy as np
from collections import Counter
import nltk
import time
import re
from itertools import chain
import numpy as np
import collections
import time
import os
import sys

In [4]:
def load_data(file_dir):
  start_time =time.time()
  f = open(file_dir, 'r')
  sentences = []
  while True:
    sentence = f.readline()
    if not sentence:
      break

    sentence = sentence.strip().lower()
    sentences.append(sentence)

  f.close()
  print("Loaded %d sentences from files, time %.2fs" % (len(sentences), time.time() - start_time))
  return sentences
def char_statistics(sentences, counter):
  for sentence in sentences:
    counter.update(sentence)
  return counter

def filter_sentences_with_punct(sentences):
  def filter_sentence(sentence):
    sentence = re.sub(r"([.!?])", r" \1", sentence)
    sentence = re.sub(r"<u>|</u>", r"", sentence)
    #return re.sub(r"[^a-zA-Z.!?]+", r" ", sentence)
    return re.sub(r"[^a-zA-Z0-9.,!?\']+", r" ", sentence)
  return [filter_sentence(sentence) for sentence in sentences]

def filter_sentences_without_punct(sentences):
  def filter_sentence(sentence):
    sentence = re.sub(r"<u>|</u>", r"", sentence)
    return re.sub(r"[^a-zA-Z0-9\']+", r" ", sentence)
  return [filter_sentence(sentence) for sentence in sentences]

def tokenizer(sentence):
  return nltk.word_tokenize(sentence)

def build_vocab(sentences, max_words=None):
  print_out("Buildding vocabulary...")
  word_count = Counter()
  for sentence in sentences:
    for word in sentence.split(" "):
      word_count[word] += 1

  print_out("The dataset has %d different words totally" % len(word_count))
  if not max_words:
    max_words = len(word_count)
  filter_out_words = len(word_count) - max_words

  word_dict = word_count.most_common(max_words)
  index2word = ["<unk>"] + ["<s>"] + ["</s>"] + [word[0] for word in word_dict]
  word2index = dict([(word, idx) for idx, word in enumerate(index2word)])

  print_out("%d words filtered out of the vocabulary and %d words in the vocabulary" % (filter_out_words, max_words))
  return index2word, word2index

def build_vocab_with_nltk(sentences, max_words=None):
  print_out("Buildding vocabulary...")
  word_count = Counter()
  for sentence in sentences:
    for word in tokenizer(sentence):
      word_count[word] += 1

  print_out("The dataset has %d different words totally" % len(word_count))
  if not max_words:
    max_words = len(word_count)
  filter_out_words = len(word_count) - max_words

  word_dict = word_count.most_common(max_words)
  index2word = ["<unk>"] + ["<s>"] + ["</s>"] + [word[0] for word in word_dict]
  word2index = dict([(word, idx) for idx, word in enumerate(index2word)])

  print_out("%d words filtered out of the vocabulary and %d words in the vocabulary" % (filter_out_words, max_words))
  return index2word, word2index

def print_out(s, f=None, new_line=True):
  if isinstance(s, bytes):
    s = s.decode("utf-8")

  if f:
    f.write(s.encode("utf-8"))
    if new_line:
      f.write(b"\n")

  # stdout
  out_s = s.encode("utf-8")
  if not isinstance(out_s, str):
    out_s = out_s.decode("utf-8")
  print out_s,

  if new_line:
    sys.stdout.write("\n")
  sys.stdout.flush()

def load_data(file_dir):
  print_out("Loading data files.")
  start_time =time.time()
  f = open(file_dir, 'r')
  sentences = []
  while True:
    sentence = f.readline()
    if not sentence:
      break

    sentence = sentence.strip().lower()
    sentences.append(sentence)

  f.close()
  print_out("Loaded %d sentences from files, time %.2fs" % (len(sentences), time.time() - start_time))
  return sentences

def filter_sentences(sentences, whitelist):
   def filter_sentence(sentence, whitelist):
     return "".join([ch for ch in sentence if ch in whitelist])

   return [filter_sentence(sentence, whitelist) for sentence in sentences] 

def build_worddict_test(sentences):
  word_count = Counter()
  for sentence in sentences:
    for word in sentence.split(" "):
      word_count[word] += 1
  return word_count

def build_worddict_test_1(sentences):
  word_count = Counter()
  for sentence in sentences:
    for word in tokenizer(sentence):
      word_count[word] += 1
  return word_count

In [5]:
train_data = load_data("train.txt")
counter = Counter()
char_counter = char_statistics(train_data, counter)
print char_counter.items()
chat_data = load_data("train.txt")

Loading data files.
 Loaded 274270 sentences from files, time 0.22s
 [('\x93', 29), ('\x97', 1053), (' ', 2640874), ('\xa3', 8), ('$', 156), ('\xab', 1), (',', 150146), ('0', 2188), ('\xb3', 1), ('4', 713), ('\xb7', 1), ('8', 545), ('<', 6683), ('\xc7', 1), ('\xd3', 1), ('\xdf', 1), ('`', 57), ('d', 401732), ('\xe7', 6), ('h', 645961), ('l', 458108), ('\xef', 1), ('p', 158704), ('\xf3', 8), ('t', 1058170), ('x', 13916), ('\xfb', 2), ('|', 49), ('\x8c', 6), ('\x94', 25), ('#', 19), ("'", 190799), ('+', 30), ('/', 3488), ('3', 740), ('7', 551), (';', 1377), ('?', 101644), ('\xc8', 4), ('\xd4', 2), ('[', 324), ('_', 123), ('\xe0', 9), ('c', 233355), ('\xe4', 26), ('g', 261967), ('\xe8', 15), ('k', 159093), ('o', 1005373), ('s', 633793), ('w', 308988), ('{', 9), ('\xfc', 2), ('\x85', 84), ('\x91', 17), ('"', 8909), ('\xa5', 1), ('&', 238), ('*', 1054), ('\xad', 2), ('.', 442255), ('2', 1060), ('6', 521), ('\xb9', 11), (':', 1551), ('>', 6685), ('\xc9', 2), ('\xd5', 1), ('^', 2), ('\xe1', 5

In [6]:
chat_data_1 = filter_sentences_with_punct(chat_data)
chat_data_2 = filter_sentences_without_punct(chat_data)
chat_data_3 = filter_sentences(chat_data, '0123456789abcdefghijklmnopqrstuvwxyz\' ')

In [7]:
vocab_11 = build_vocab(chat_data_1)
vocab_12 = build_vocab_with_nltk(chat_data_1)
vocab_21 = build_vocab(chat_data_2)
vocab_22 = build_vocab_with_nltk(chat_data_2)
vocab_31 = build_vocab(chat_data_3)
vocab_32 = build_vocab_with_nltk(chat_data_3)

Buildding vocabulary...
 The dataset has 67370 different words totally
 0 words filtered out of the vocabulary and 67370 words in the vocabulary
 Buildding vocabulary...
 The dataset has 49303 different words totally
 0 words filtered out of the vocabulary and 49303 words in the vocabulary
 Buildding vocabulary...
 The dataset has 52088 different words totally
 0 words filtered out of the vocabulary and 52088 words in the vocabulary
 Buildding vocabulary...
 The dataset has 47860 different words totally
 0 words filtered out of the vocabulary and 47860 words in the vocabulary
 Buildding vocabulary...
 The dataset has 64860 different words totally
 0 words filtered out of the vocabulary and 64860 words in the vocabulary
 Buildding vocabulary...
 The dataset has 60039 different words totally
 0 words filtered out of the vocabulary and 60039 words in the vocabulary



In [8]:
word_dict_11 = build_worddict_test(chat_data_1)
word_dict_12 = build_worddict_test_1(chat_data_1)
word_dict_21 = build_worddict_test(chat_data_2)
word_dict_22 = build_worddict_test_1(chat_data_2)
word_dict_31 = build_worddict_test(chat_data_3)
word_dict_32 = build_worddict_test_1(chat_data_3)

In [9]:
word_item_11 = word_dict_11.items()
word_item_12 = word_dict_12.items()
word_item_21 = word_dict_21.items()
word_item_22 = word_dict_22.items()
word_item_31 = word_dict_31.items()
word_item_32 = word_dict_32.items()

In [10]:
word_list_11 = word_dict_11.most_common(50000)
word_list_12 = word_dict_12.most_common(50000)
word_list_21 = word_dict_21.most_common(50000)
word_list_22 = word_dict_22.most_common(50000)
word_list_31 = word_dict_31.most_common(50000)
word_list_32 = word_dict_32.most_common(50000)

In [11]:
print word_item_12[:10]

[('nunnery', 1), ('woods', 84), ('clotted', 2), ('spiders', 5), ('hanging', 145), ('woody', 30), ('localized', 2), ('spidery', 1), ('sevens', 2), ('disobeying', 1)]


In [13]:
print word_list_12[4950:5000]

[('lungs', 28), ('dignity', 28), ('suffered', 28), ('hotels', 28), ('terrance', 28), ('awkward', 28), ('gag', 28), ('savings', 28), ('environment', 28), ('crashed', 28), ('underneath', 28), ('ammo', 28), ('nails', 28), ('calvin', 28), ('unlike', 28), ('batteries', 28), ('remembering', 28), ('yer', 28), ('sophie', 28), ('.with', 28), ('frightening', 28), ('approve', 28), ('haven', 28), ('lazy', 28), ('mayo', 28), ('bravo', 28), ('assistance', 28), ('marked', 28), ('leather', 28), ('lip', 28), ('presents', 28), ('differently', 28), ('panties', 28), ('ministry', 28), ('parent', 28), ('investigating', 28), ('gino', 28), ('fink', 28), ('follows', 28), ('expedition', 28), ('negro', 28), ('gather', 28), ('fortress', 28), ('arizona', 28), ('exclusive', 28), ('relieved', 28), ('wichita', 28), ('sittin', 28), ('debate', 28), ('buffy', 28)]
